In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Football Analytics Visualizations\n",
    "## Complete Chart Gallery\n",
    "\n",
    "This notebook showcases all visualizations used in the Football Alpha Analysis project."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from adjustText import adjust_text\n",
    "import sys\n",
    "sys.path.append('../src')\n",
    "from analysis import get_data\n",
    "\n",
    "plt.style.use('seaborn-v0_8-whitegrid')\n",
    "df = get_data()\n",
    "print(f\"Loaded {len(df)} players\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Scatter Plot: xG vs Actual Goals"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(14, 10))\n",
    "scatter = plt.scatter(df['xg'], df['gls'], alpha=0.5, c=df['finishing_alpha'], \n",
    "                      cmap='RdYlGn', s=50, edgecolors='white', linewidth=0.5)\n",
    "plt.colorbar(scatter, label='Finishing Alpha')\n",
    "\n",
    "# Perfect conversion line\n",
    "max_val = max(df['xg'].max(), df['gls'].max())\n",
    "plt.plot([0, max_val], [0, max_val], 'k--', linewidth=2, label='Perfect Conversion')\n",
    "\n",
    "# Label outliers\n",
    "top = df.nlargest(10, 'finishing_alpha')\n",
    "worst = df.nsmallest(5, 'finishing_alpha')\n",
    "outliers = pd.concat([top, worst])\n",
    "\n",
    "texts = [plt.text(row['xg'], row['gls'], row['player'], fontsize=9, fontweight='bold') \n",
    "         for _, row in outliers.iterrows()]\n",
    "adjust_text(texts, arrowprops=dict(arrowstyle='-', color='gray', lw=0.5))\n",
    "\n",
    "plt.xlabel('Expected Goals (xG)', fontsize=12)\n",
    "plt.ylabel('Actual Goals', fontsize=12)\n",
    "plt.title('Expected Goals vs Actual Goals (2025-26 Season)', fontsize=14)\n",
    "plt.legend(loc='upper left')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Horizontal Bar Chart: Top Finishers"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "fig, axes = plt.subplots(1, 2, figsize=(16, 8))\n",
    "\n",
    "# Top 15 clinical finishers\n",
    "top15 = df.nlargest(15, 'finishing_alpha')\n",
    "colors1 = ['#2ecc71' for _ in range(15)]\n",
    "axes[0].barh(top15['player'], top15['finishing_alpha'], color=colors1)\n",
    "axes[0].set_xlabel('Finishing Alpha')\n",
    "axes[0].set_title('Top 15 Clinical Finishers')\n",
    "axes[0].invert_yaxis()\n",
    "\n",
    "# Bottom 15 finishers\n",
    "worst15 = df.nsmallest(15, 'finishing_alpha')\n",
    "colors2 = ['#e74c3c' for _ in range(15)]\n",
    "axes[1].barh(worst15['player'], worst15['finishing_alpha'], color=colors2)\n",
    "axes[1].set_xlabel('Finishing Alpha')\n",
    "axes[1].set_title('Top 15 Underperforming Finishers')\n",
    "axes[1].invert_yaxis()\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. League Comparison: Grouped Bar Chart"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "league_stats = df.groupby('comp').agg({\n",
    "    'finishing_alpha': 'mean',\n",
    "    'playmaking_alpha': 'mean'\n",
    "}).round(3)\n",
    "\n",
    "fig, axes = plt.subplots(1, 2, figsize=(14, 5))\n",
    "\n",
    "for i, (col, title) in enumerate([('finishing_alpha', 'Finishing'), ('playmaking_alpha', 'Playmaking')]):\n",
    "    order = league_stats.sort_values(col).index\n",
    "    colors = ['#2ecc71' if league_stats.loc[l, col] > 0 else '#e74c3c' for l in order]\n",
    "    axes[i].barh(order, league_stats.loc[order, col], color=colors)\n",
    "    axes[i].axvline(x=0, color='black', linewidth=0.5)\n",
    "    axes[i].set_xlabel(f'Average {title} Alpha')\n",
    "    axes[i].set_title(f'League {title} Efficiency')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Shot Conversion Analysis: Scatter Plot"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "shooters = df[df['sh'] >= 10].copy()\n",
    "shooters['shot_accuracy'] = (shooters['sot'] / shooters['sh']) * 100\n",
    "shooters['conversion_rate'] = (shooters['gls'] / shooters['sh']) * 100\n",
    "\n",
    "plt.figure(figsize=(12, 10))\n",
    "scatter = plt.scatter(shooters['shot_accuracy'], shooters['conversion_rate'],\n",
    "                      alpha=0.6, c=shooters['gls'], cmap='YlOrRd', s=60)\n",
    "plt.colorbar(scatter, label='Total Goals')\n",
    "\n",
    "# Label top converters\n",
    "top_conv = shooters.nlargest(10, 'conversion_rate')\n",
    "texts = [plt.text(row['shot_accuracy'], row['conversion_rate'], row['player'], \n",
    "                  fontsize=9, fontweight='bold') for _, row in top_conv.iterrows()]\n",
    "adjust_text(texts, arrowprops=dict(arrowstyle='-', color='gray', lw=0.5))\n",
    "\n",
    "plt.xlabel('Shot Accuracy (% on target)', fontsize=12)\n",
    "plt.ylabel('Conversion Rate (% goals per shot)', fontsize=12)\n",
    "plt.title('Shot Accuracy vs Goal Conversion Rate', fontsize=14)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Radar Chart: Player Comparison"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Compare top 2 scorers\n",
    "top_scorers = df.nlargest(2, 'gls')\n",
    "p1 = top_scorers.iloc[0]\n",
    "p2 = top_scorers.iloc[1]\n",
    "\n",
    "metrics = ['gls', 'ast', 'xg', 'xag', 'finishing_alpha', 'playmaking_alpha']\n",
    "max_vals = {m: df[m].max() for m in metrics}\n",
    "\n",
    "fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))\n",
    "\n",
    "angles = np.linspace(0, 2 * np.pi, len(metrics), endpoint=False).tolist()\n",
    "angles += angles[:1]\n",
    "\n",
    "# Player 1\n",
    "vals1 = [(p1[m] / max_vals[m] * 100) if max_vals[m] != 0 else 0 for m in metrics]\n",
    "vals1 += vals1[:1]\n",
    "ax.plot(angles, vals1, 'o-', linewidth=2, label=p1['player'], color='#3498db')\n",
    "ax.fill(angles, vals1, alpha=0.25, color='#3498db')\n",
    "\n",
    "# Player 2\n",
    "vals2 = [(p2[m] / max_vals[m] * 100) if max_vals[m] != 0 else 0 for m in metrics]\n",
    "vals2 += vals2[:1]\n",
    "ax.plot(angles, vals2, 'o-', linewidth=2, label=p2['player'], color='#e74c3c')\n",
    "ax.fill(angles, vals2, alpha=0.25, color='#e74c3c')\n",
    "\n",
    "ax.set_xticks(angles[:-1])\n",
    "ax.set_xticklabels(metrics)\n",
    "ax.set_ylim(0, 100)\n",
    "ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))\n",
    "ax.set_title(f\"{p1['player']} vs {p2['player']}\", fontsize=14)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Pie Chart: Squad Composition"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_pos(p):\n",
    "    if pd.isna(p): return 'Unknown'\n",
    "    p = p.upper()\n",
    "    if 'GK' in p: return 'GK'\n",
    "    elif 'DF' in p: return 'DF'\n",
    "    elif 'MF' in p: return 'MF'\n",
    "    elif 'FW' in p: return 'FW'\n",
    "    return 'Unknown'\n",
    "\n",
    "# Example: Real Madrid\n",
    "team = df[df['squad'].str.contains('Real Madrid', case=False)]\n",
    "pos_counts = team['pos'].apply(get_pos).value_counts()\n",
    "\n",
    "plt.figure(figsize=(8, 8))\n",
    "colors = {'GK': '#9b59b6', 'DF': '#3498db', 'MF': '#2ecc71', 'FW': '#e74c3c'}\n",
    "plt.pie(pos_counts.values, labels=pos_counts.index, autopct='%1.0f%%',\n",
    "        colors=[colors.get(p, '#95a5a6') for p in pos_counts.index], startangle=90)\n",
    "plt.title(f\"Real Madrid - Squad Composition\", fontsize=14)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Stacked Bar Chart: Goal Contributions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Top 15 by G+A\n",
    "df_copy = df.copy()\n",
    "df_copy['g_a'] = df_copy['gls'] + df_copy['ast']\n",
    "top15_ga = df_copy.nlargest(15, 'g_a').sort_values('g_a', ascending=True)\n",
    "\n",
    "plt.figure(figsize=(12, 8))\n",
    "plt.barh(top15_ga['player'], top15_ga['gls'], label='Goals', color='#3498db')\n",
    "plt.barh(top15_ga['player'], top15_ga['ast'], left=top15_ga['gls'], label='Assists', color='#2ecc71')\n",
    "plt.xlabel('Goal Contributions (G+A)')\n",
    "plt.title('Top 15 Players by Goal Contributions', fontsize=14)\n",
    "plt.legend()\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Heatmap: Correlation Matrix"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "key_metrics = ['gls', 'ast', 'xg', 'xag', 'finishing_alpha', 'playmaking_alpha', \n",
    "               'sh', 'sot', 'kp', 'ppa', 'col_90s']\n",
    "corr = df[key_metrics].corr()\n",
    "\n",
    "plt.figure(figsize=(12, 10))\n",
    "sns.heatmap(corr, annot=True, cmap='RdYlGn', center=0, fmt='.2f',\n",
    "            square=True, linewidths=0.5)\n",
    "plt.title('Correlation Matrix - Key Performance Metrics', fontsize=14)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. Box Plot: Goals by Position"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "df['main_pos'] = df['pos'].apply(get_pos)\n",
    "\n",
    "fig, axes = plt.subplots(1, 2, figsize=(14, 6))\n",
    "\n",
    "# Goals by position\n",
    "df.boxplot(column='gls', by='main_pos', ax=axes[0])\n",
    "axes[0].set_title('Goals Distribution by Position')\n",
    "axes[0].set_xlabel('Position')\n",
    "axes[0].set_ylabel('Goals')\n",
    "\n",
    "# Finishing Alpha by position\n",
    "df.boxplot(column='finishing_alpha', by='main_pos', ax=axes[1])\n",
    "axes[1].axhline(y=0, color='red', linestyle='--')\n",
    "axes[1].set_title('Finishing Alpha by Position')\n",
    "axes[1].set_xlabel('Position')\n",
    "axes[1].set_ylabel('Finishing Alpha')\n",
    "\n",
    "plt.suptitle('')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 10. Minutes vs Goals Scatter"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(12, 10))\n",
    "scatter = plt.scatter(df['min'], df['gls'], alpha=0.5, c=df['gls_per90'], \n",
    "                      cmap='YlOrRd', s=50)\n",
    "plt.colorbar(scatter, label='Goals per 90')\n",
    "\n",
    "# Label top scorers\n",
    "top = df.nlargest(10, 'gls')\n",
    "texts = [plt.text(row['min'], row['gls'], row['player'], fontsize=9, fontweight='bold')\n",
    "         for _, row in top.iterrows()]\n",
    "adjust_text(texts, arrowprops=dict(arrowstyle='-', color='gray', lw=0.5))\n",
    "\n",
    "plt.xlabel('Minutes Played', fontsize=12)\n",
    "plt.ylabel('Goals', fontsize=12)\n",
    "plt.title('Minutes Played vs Goals Scored', fontsize=14)\n",
    "plt.gca().yaxis.set_major_locator(plt.MaxNLocator(integer=True))\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Summary\n",
    "\n",
    "This notebook demonstrated 10 different visualization types:\n",
    "\n",
    "1. **Scatter Plot** - xG vs Goals with color mapping\n",
    "2. **Horizontal Bar Chart** - Top/worst finishers\n",
    "3. **Grouped Bar Chart** - League comparison\n",
    "4. **Shot Conversion Scatter** - Accuracy vs conversion\n",
    "5. **Radar Chart** - Player comparison\n",
    "6. **Pie Chart** - Squad composition\n",
    "7. **Stacked Bar Chart** - Goal contributions\n",
    "8. **Heatmap** - Correlation matrix\n",
    "9. **Box Plot** - Distribution by category\n",
    "10. **Minutes vs Goals** - Playing time analysis"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.9.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}